# PPA3 Database Table Viewer
If PPA is live and people are using it, and you want to view tables, please do so through this tool rather than Arc Pro. Viewing in Pro will create a schema lock on the tables, so when a user runs the tool, it will cause an error because it cannot log the run output to the locked database tables.

This viewer gets around this issue by creating an in-memory copy of desired gdb tables.

In [1]:
import pandas as pd

import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

file_gdb = r'\\arcserver-svr\D\PPA3_SVR\PPA3_GIS_SVR\PPA3_run_data.gdb'
arcpy.env.workspace = file_gdb

fc_pmaster = 'project_master'
f_uid = 'project_uid'

df_pmaster = pd.DataFrame.spatial.from_featureclass(fc_pmaster)

def esri_object_to_df(in_esri_obj, esri_obj_fields, index_field=None):
    '''converts esri gdb table, feature class, feature layer, or SHP to pandas dataframe'''
    data_rows = []
    with arcpy.da.SearchCursor(in_esri_obj, esri_obj_fields) as cur:
        for row in cur:
            out_row = list(row)
            data_rows.append(out_row)

    out_df = pd.DataFrame(data_rows, index=index_field, columns=esri_obj_fields)
    return out_df

print([t for t in arcpy.ListTables()])


['rp_artexp_vmt', 'rp_artexp_econ', 'rp_artexp_eq', 'rp_artexp_mm', 'rp_artexp_sgr', 'rp_fwy_vmt', 'rp_fwy_cong', 'rp_fwy_mm', 'rp_fwy_econ', 'rp_fwy_frgt', 'rp_fwy_saf', 'rp_artsgr_sgr', 'cd_compactdev', 'cd_mixeduse', 'cd_houschoice', 'cd_naturpres', 'rp_artexp_cong', 'rp_artexp_frgt', 'rp_artexp_saf', 'cd_trnchoice', 'cd_existgasset', 'TEST_TABLE']


## View of most recent entries into master table

In [5]:
fields = ['proj_name','juris','proj_type', 'project_uid', 'comm_type','time_created','user_email']
df_pmaster[fields].sort_values(by='time_created', ascending=False).head(10)

,proj_name,juris,proj_type,project_uid,comm_type,time_created,user_email
287,Enterprise Corridor and Bridge Crossing,City of West Sacramento,Arterial or Transit Expansion,d6dbad2f-3cb2-4036-aaee-c1d9cd5a1b90,Established Communities,2023-02-02 10:06:34.000000,mccoyj@cityofwestsacramento.org
286,Sacramento International Airport Station Shuttle,San Joaquin Regional Rail Commission,Arterial or Transit Expansion,b5e43016-4cb1-437d-a5fa-eebf3a9cd4d4,Ag,2023-02-02 10:03:28.000000,rbissegger@markthomas.com
285,Sacramento River Parkway Supplimentary Informa...,City of Sacramento,Arterial State of Good Repair,9f463c6b-f45c-4e43-8f65-ae5c263a490c,Established Communities,2023-02-01 11:38:26.000000,jmkragh@cityofsacramento.org
284,Routier,Rancho Cordova,Arterial or Transit Expansion,4fc85163-375b-4cae-8ed3-4b014f00f6ed,Established Communities,2023-02-01 11:32:18.000000,dwhittington@dewberry.com
283,Mather,Rancho Cordova,Arterial or Transit Expansion,752e4795-23aa-4240-a422-acc4138e625d,Arterials & Suburban Corridors,2023-02-01 11:29:54.000001,dwhittington@dewberry.com
282,Old Placerville Rd,Rancho Cordova,Freeway Expansion,b07ba1ad-5ad0-472a-b463-01c12971fbf2,Established Communities,2023-02-01 10:28:22.000000,dwhittington@dewberry.com
281,Sunrise,Sunrise,Arterial or Transit Expansion,9b476db8-c01a-4202-a91e-9ce9bb7a85a0,Arterials & Suburban Corridors,2023-02-01 10:01:32.000000,dwhittington@dewberry.com
280,rancho,rancho,Arterial or Transit Expansion,c2822400-cb26-4920-b6d6-fb5c5b663faa,Arterials & Suburban Corridors,2023-02-01 09:43:38.000000,dwhittington@dewberry.com
279,Hazel Avenue Interchange,Sacramento County,Arterial or Transit Expansion,406f1507-21ff-4785-892e-29a04f5f8af4,Arterials & Suburban Corridors,2023-02-01 08:27:00.000000,rbissegger@markthomas.com
278,Hazel Avenue IC,Sacramento County,Arterial or Transit Expansion,44b4d489-d467-4c7b-afe1-67aca7821493,Arterials & Suburban Corridors,2023-02-01 08:20:35.000000,rbissegger@markthomas.com


## Find UID values that are in any of the data tables but not in the master table

In [3]:


def get_uids(in_tbl):
    uids = []

    try:
        with arcpy.da.SearchCursor(in_tbl, field_names=[f_uid]) as cur:
            for row in cur:
                uids.append(row[0])
    except:
        print(f"{in_tbl} does not have field {f_uid}. Skipping...")
        
    return uids

data_tables = [t for t in arcpy.ListTables()]
pmaster_uids = df_pmaster[f_uid].to_list()

out_dict = {}

fake_id = 'UID_NOT_FOUND' # placeholder ID used when running tool locally for testing

for t in data_tables:
    tbl_uids = get_uids(t)
    
    uids_not_in_master = []
    for uid in tbl_uids:
        if uid not in pmaster_uids and uid != fake_id:
            uids_not_in_master.append(uid)
            
    out_dict[t] = uids_not_in_master
    if len(out_dict[t]) > 0:
        print(f"table {t} has the follwing UIDs not in the project_master table: {out_dict[t]}")
        
total_missing = sum([len(v) for tbl, v in out_dict.items()])
if total_missing == 0: print("no IDs found in subreport tables that are not also in project_master")
    

TEST_TABLE does not have field project_uid. Skipping...
no IDs found in subreport tables that are not also in project_master


## Show specific subreport data and for specific projects

In [7]:

subrpt_tbl = 'rp_artexp_cong'
project_uid = 'b5e43016-4cb1-437d-a5fa-eebf3a9cd4d4'

fields = [f.name for f in arcpy.ListFields(subrpt_tbl)]

#uncomment if you want to filter which fields appear
# fields = [f_uid, 'crash_cnt', 'crash_100mvmt'] # [f.name for f in arcpy.ListFields(subrpt_tbl)]

df_master_fields = [f_uid, 'time_created', 'user_email', 'len_mi', 'aadt']
df_subrpt = esri_object_to_df(subrpt_tbl, esri_obj_fields=fields, index_field=None)


df_jn = df_pmaster[df_master_fields].merge(df_subrpt, on=f_uid, suffixes=('','_y')) \
    .sort_values(by='time_created', ascending=False)

if project_uid:
    df = df_jn.loc[df_jn['project_uid'] == project_uid].head()
else:
    df = df_jn.head(10)
    
df

,project_uid,time_created,user_email,len_mi,aadt,OBJECTID,aadt_y,congspd_nb,congspd_sb,congspd_eb,...,jobs_base,jobs_future,du_base,du_future,congspd_wrst,congrat_wrst,lottr_ampk_wrst,lottr_midday_wrst,lottr_pmpk_wrst,lottr_wknd_wrst
131,b5e43016-4cb1-437d-a5fa-eebf3a9cd4d4,2023-02-02 10:03:28,rbissegger@markthomas.com,37.13464,0.0,300,0,33.49519,35.596416,NaN,...,59704.089844,74169.820312,40781.0,52714.289062,33.49519,0.672056,1.286697,1.285558,1.233037,1.277119


In [42]:
df_jn.loc[df_jn.crash_100mvmt > 100].head()

,project_uid,time_created,user_email,len_mi,aadt,crash_cnt,crash_100mvmt
21,55160fbd-9617-4c09-a05e-da8819a2b71c,2022-12-06 13:47:37,jhong@sacog.org,3.670994,5.0,136,463089.843750
45,fbc41452-2ed7-44ce-b1d1-3563bad0f73c,2023-01-17 06:06:46,rbissegger@markthomas.com,1.861210,58600.0,36,130.566345
